In [7]:
import numpy as np
from dataclasses import dataclass

# ==============================
#  UTILITÁRIOS
# ==============================

@dataclass
class Node:
    name: str
    x: float
    y: float

@dataclass
class Member:
    name: str
    n1: int  # índice do nó i
    n2: int  # índice do nó j
    A: float # área (qualquer valor positivo; para forças internas, o valor não altera o sinal)
    E: float # módulo de elasticidade (idem observação acima)

def dof_index(i):  # mapeia nós -> DOFs globais [ux, uy]
    return [2*i, 2*i+1]

def bar_local_stiffness(xi, yi, xj, yj, A, E):
    L = np.hypot(xj - xi, yj - yi)
    c = (xj - xi)/L
    s = (yj - yi)/L
    k = (A*E/L) * np.array([[ c*c,  c*s, -c*c, -c*s],
                            [ c*s,  s*s, -c*s, -s*s],
                            [-c*c, -c*s,  c*c,  c*s],
                            [-c*s, -s*s,  c*s,  s*s]])
    return k, L, c, s

def assemble_global_stiffness(nodes, members):
    ndofs = 2*len(nodes)
    K = np.zeros((ndofs, ndofs))
    for m in members:
        i, j = m.n1, m.n2
        xi, yi = nodes[i].x, nodes[i].y
        xj, yj = nodes[j].x, nodes[j].y
        k, *_ = bar_local_stiffness(xi, yi, xj, yj, m.A, m.E)
        dofi, dofj = dof_index(i), dof_index(j)
        idx = dofi + dofj
        for a in range(4):
            for b in range(4):
                K[idx[a], idx[b]] += k[a, b]
    return K

def solve_truss(nodes, members, loads, fixed_dofs):
    """
    loads: vetor [2*N] com forças nodais (Fx,Fy) no sistema global
    fixed_dofs: lista com índices de DOFs travados (0-based)
    """
    K = assemble_global_stiffness(nodes, members)
    all_dofs = np.arange(K.shape[0])
    free = np.setdiff1d(all_dofs, np.array(fixed_dofs, dtype=int))
    # resolve
    Kg = K[np.ix_(free, free)]
    Fg = loads[free]
    ug = np.zeros(K.shape[0])
    ug[free] = np.linalg.solve(Kg, Fg)
    # reações
    R = K @ ug - loads
    return ug, R, K

def member_forces(nodes, members, u):
    """retorna lista com (nome, N axial em lb; + tração, - compressão)"""
    out = []
    for m in members:
        i, j = m.n1, m.n2
        xi, yi = nodes[i].x, nodes[i].y
        xj, yj = nodes[j].x, nodes[j].y
        k, L, c, s = bar_local_stiffness(xi, yi, xj, yj, m.A, m.E)
        dofi, dofj = dof_index(i), dof_index(j)
        ue = np.array([u[dofi[0]], u[dofi[1]], u[dofj[0]], u[dofj[1]]])
        # esforço axial: N = AE/L * [-c -s c s] * ue
        N = (m.A*m.E/L) * np.array([-c, -s, c, s]) @ ue
        out.append((m.name, N))
    return out

def pretty_table(headers, rows, fmt=None):
    if fmt is None:
        fmt = ["{}"]*len(headers)
    colw = [max(len(h), max(len(fmt[i].format(r[i])) for r in rows)) for i,h in enumerate(headers)]
    line = " | ".join(h.ljust(colw[i]) for i,h in enumerate(headers))
    sep  = "-+-".join("-"*colw[i] for i in range(len(headers)))
    print(line); print(sep)
    for r in rows:
        print(" | ".join(fmt[i].format(r[i]).rjust(colw[i]) for i in range(len(headers))))

# ======================================================
#  GEOMETRIA E CARREGAMENTOS (AJUSTE AQUI)
#  — tentativa coerente com a figura enviada —
#  Unidades: ft, lb (E e A podem ser 1.0 — só afetam deslocamentos,
#  não os sinais das forças internas/reações)
# ======================================================

# Nós (na ordem: A, B, C, D, E)
# A e B no banzo superior (intervalo 12 ft), E e C no banzo inferior,
# D é nó intermediário (metade do vão e metade da altura).
nodes = [
    Node("A", 0.0,   0.0),   # A
    Node("B", 12.0,  0.0),   # B
    Node("C", 24.0, 0),  # C (apoio pino)
    Node("D", 6.0,  -8.0),   # D (nó interno)
    Node("E", 18.0,  -8.0),  # E (apoio rolete)
]

# Barras (ligação em índice dos nós acima)
# Esqueleto "K": banzo sup. AB; banzo inf. EC; montantes/diagonais: AD, DB, DE, EB, BC
E = 1.0  # módulo de elasticidade fictício
A = 1.0  # área fictícia
members = [
    Member("AB", 0, 1, A, E),
    Member("AD", 0, 3, A, E),
    Member("DB", 3, 1, A, E),
    Member("DE", 3, 4, A, E),
    Member("EB", 4, 1, A, E),
    Member("EC", 4, 2, A, E),
    Member("BC", 1, 2, A, E),
]

# Cargas nodais (Fx,Fy) em lb. Sinal positivo: +x à direita, +y para cima.
N = len(nodes)
F = np.zeros(2*N)
# 2000 lb para baixo em A; 1000 lb para baixo em B
F[2*0+1] = -2000.0
F[2*1+1] = -1000.0

# Restrições (apoios):
# C pino -> ux=0, uy=0 ; E rolete vertical -> uy=0
fixed_dofs = []
# C (índice 2)
fixed_dofs += dof_index(2)           # trava ux, uy em C
# E (índice 4) — rolete vertical: uy = 0 (ux livre)
fixed_dofs += [2*4+1]

# ======================================================
#  SOLUÇÃO
# ======================================================

u, R, K = solve_truss(nodes, members, F, fixed_dofs)

# Tabelas: deslocamentos, reações e esforço axial em cada barra
disp_rows = []
for i, nd in enumerate(nodes):
    disp_rows.append([nd.name, u[2*i], u[2*i+1]])
print("\nDESLOCAMENTOS (unid. arbitrária pois A e E são fictícios)")
pretty_table(["Nó", "ux", "uy"], disp_rows, ["{}", "{: .6e}", "{: .6e}"])

reac_rows = []
for i, nd in enumerate(nodes):
    Rx, Ry = R[2*i], R[2*i+1]
    if abs(Rx) > 1e-6 or abs(Ry) > 1e-6:
        reac_rows.append([nd.name, Rx, Ry])
print("\nREAÇÕES DE APOIO (lb)  [sinal + para direita/cima]")
pretty_table(["Nó", "Rx", "Ry"], reac_rows, ["{}", "{: .2f}", "{: .2f}"])

mforces = member_forces(nodes, members, u)
m_rows = []
for name, Nax in mforces:
    m_rows.append([name, Nax, "Tração" if Nax>=0 else "Compressão"])
print("\nFORÇAS NAS BARRAS (lb)  [Tração (+), Compressão (–)]")
pretty_table(["Barra", "N (lb)", "Tipo"], m_rows, ["{}", "{: .2f}", "{}"])



DESLOCAMENTOS (unid. arbitrária pois A e E são fictícios)
Nó | ux            | uy           
---+---------------+--------------
 A | -8.100000e+04 | -6.467500e+05
 B | -6.300000e+04 | -2.035000e+05
 C |  0.000000e+00 |  0.000000e+00
 D |  1.818333e+05 | -4.183750e+05
 E |  1.458333e+05 |  0.000000e+00

REAÇÕES DE APOIO (lb)  [sinal + para direita/cima]
Nó | Rx    | Ry       
---+-------+----------
 C |  0.00 |  -7000.00
 E | -0.00 |  10000.00

FORÇAS NAS BARRAS (lb)  [Tração (+), Compressão (–)]
Barra | N (lb)   | Tipo      
------+----------+-----------
   AB |  1500.00 |     Tração
   AD | -2500.00 | Compressão
   DB |  2500.00 |     Tração
   DE | -3000.00 | Compressão
   EB | -3750.00 | Compressão
   EC | -8750.00 | Compressão
   BC |  5250.00 |     Tração
